In [1]:
#------------------------------------------------------------------------------------------------------------
# huggingface text2text-generation 모델을 연동한 langchaing  Q&A 예제
# -> 로컬 문서를 임베딩 하고, 유사한 문서를 찾은후, LM(Language Moel) 으로 문서와 쿼리를 날려서 응답결과를 얻는 예시임.
#
# 출처 : https://medium.com/the-techlife/using-huggingface-openai-and-cohere-models-with-langchain-db57af14ac5b
# 참고 : https://python.langchain.com/en/latest/
#
# pip install 'huggingface_hub'
# pip install unstructured      -> DirectoryLoader (파일 로딩할때)
#
# => 결론 : hugggingface LM(Language Moel)은 실패. 따라서 임베딩만 huggingface 모델 이용하고, LM은 OPENAI 모델 이용함
#------------------------------------------------------------------------------------------------------------

from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from huggingface_hub import hf_hub_download
import textwrap
import glob

# hugging face api_key 입력 
# => https://huggingface.co/settings/tokens 확인 가능
# HUGGING_FACE_API_KEY = 'hf_xxxx'

# openai 모델 사용시에는 아래처럼 api_key 설정함.
from langchain.llms import OpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxx"

In [2]:
# Prompt 생성 
# -> 목적에 맞게 되도록이면 영문으로 프롬프트 작성함.
# Please try to give me the most beneficial answers to my question with reasoning for why they are correct.(제 질문에 가장 유익한 답변을 해주세요. 그들이 올바른 이유에 대한 추론과 질문)

template = """ You are going to be my assistant.
Please provide a brief, short-answer response to the question that best suits you.
Question: {input} Answer: """

prompt = PromptTemplate(template=template, input_variables=["input"])

In [3]:
# text2text 모델과 langchain 연결
'''
# huggingface 모델 연결
model = HuggingFaceHub(repo_id="facebook/mbart-large-50",
                       model_kwargs={"temperature": 0, "max_length": 200},
                       huggingfacehub_api_token=HUGGING_FACE_API_KEY)
'''

# openai 모델 연결
model = OpenAI(model_name= "text-davinci-003" ) 

chain = LLMChain(prompt=prompt, llm=model)

In [4]:
# 임베딩 모델 로딩
hf_embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(
/MOCOMSYS/anaconda3/envs/bong/lib/python3.9/site-packages/huggingface_hub/file_download.py:621: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [5]:
# langchain DirectoryLoader를 이용하여 txt 데이터 불러옴.
# -> 참조: https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/file_directory.html
#
my_loader = DirectoryLoader('mydata', glob= '**/*.txt', loader_cls=TextLoader) 
docs = my_loader.load() 
print(f'len(docs):{len(docs)}')

# 청크로 나눔.
text_split = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 50 ) 
text = text_split.split_documents(docs)
print(f'len(text):{len(text)}')


len(docs):2
len(text):68


In [6]:
text[1]

Document(page_content='of freedom fighters known as the Rebel Alliance, who aim to destroy the Empire\'s newest weapon, the Death Star. Luke Skywalker becomes caught in the conflict while learning the ways of a metaphysical power known as "the Force" from Jedi Master Obi-Wan Kenobi. The cast includes Mark Hamill, Harrison Ford, Carrie Fisher, Peter Cushing, Alec Guinness, David Prowse, James Earl Jones, Anthony Daniels, Kenny Baker, and Peter Mayhew.', metadata={'source': 'mydata/starwars.txt'})

In [7]:
# 텍스트와 임베딩을 FAISS에 전달 하여 임베딩 수행
# -> 기타 FAISS 외 다른 임베딩 방법은 https://python.langchain.com/en/latest/modules/indexes/vectorstores.html 참조
#
vectorstore = FAISS.from_documents(text, hf_embeddings)

In [9]:
# 체인 만들고 쿼리 날리기.
my_chain = load_qa_with_sources_chain(model, chain_type="refine")

#query = "When did the Star Wars movies come out?"
query = "국내 출장시 1박 숙박비는 1인당 얼마인가요?"
documents = vectorstore.similarity_search(query)
print(f'len(documents):{len(documents)}')
print(documents)


len(documents):4
[Document(page_content='7.3. 국내 가족여비\n7.3.1. 국내 가족여비는 국내 이전자로서 이전할 때 가족(배우자, 본인 및 배우자의 직계존속, 직계비속으로서 생계를 같이 하는 사람을 말한다)을 동반하거나 이전 후에 가족을 불러오는 임직원에게 지급한다.\n7.3.2. 국내 이전자가 특별한 사정으로 대표이사의 허가를 받아 부임의 명을 받은 날로부터 6개월 이내에 전임지 또는 전임지 외의 지역에서 신임지 외의 지역으로 가족을 이전하는 경우에는 가족여비를 지급한다.(예를 들어 광주로 부임하는데, 가족은 광주광역시 근처의 다른 지방으로 이사하는 경우)\n7.3.3. 가족여비는 가족 1명마다 다음 각 호의 구분에 따른 금액을 지급한다.\n가. 운임과 숙박비 : 본인이 여행하는 경우에 지급하는 금액\n나. 일비와 식비 : 본인이 여행하는 경우에 지급할 수 있는 금액을 기준으로 12세 이상인 가족에 대해서는 3분의 2, 12세 미만의 가족에 대해서는 3분의 1에 상당하는 금액\n7.4. 해외 가족여비', metadata={'source': 'mydata/출장여비규정.txt'}), Document(page_content='자를 말한다.\n15. 원격지 근무\n15.1. 출장의 정의에 부합하며, 지사, 사무소가 아닌 지역으로, 회사의 비용으로 숙박시설을 마련해 주는 경우를 원격지 근무라고 한다.\n15.2.원격지 근무자에게는 숙박비, 운임, 일비, 식비, 원격지근무수당을 지급한다.\n15.3. 숙박비는 숙박시설을 회사에서 제공(숙박시설 임차 및 숙박에 필요한 기본적인 물품 제공)하는 것으로 하여 별도로 지급하지 아니한다.\n15.4. 숙박에 필요한 기본 물품 구매는 별표9. 원격지 근무시 필요 물품 구매에 따른다.\n15.5. 운임은 월 4회(왕복 8회), 근무지에서 서울역까지의 KTX요금 혹은 이에 준하는 교통편의(시외버스 혹은 고속버스) 요금을 현금으로 지급한다.\n15.6. 일비와 식비는 월 4회(왕복 8회)의 이동일을

In [12]:
# 아주 오래 걸림.(*1분 이상??)
#result = my_chain({"input_documents": documents, "question": query}, return_only_outputs=True)  # output_text만 출력할때.
result = my_chain({"input_documents": documents, "question": query})  # 모든 출력값 출력

#result = with_sources_chain({"input_documents": documents, "question": query})

In [13]:
result

{'input_documents': [Document(page_content='7.3. 국내 가족여비\n7.3.1. 국내 가족여비는 국내 이전자로서 이전할 때 가족(배우자, 본인 및 배우자의 직계존속, 직계비속으로서 생계를 같이 하는 사람을 말한다)을 동반하거나 이전 후에 가족을 불러오는 임직원에게 지급한다.\n7.3.2. 국내 이전자가 특별한 사정으로 대표이사의 허가를 받아 부임의 명을 받은 날로부터 6개월 이내에 전임지 또는 전임지 외의 지역에서 신임지 외의 지역으로 가족을 이전하는 경우에는 가족여비를 지급한다.(예를 들어 광주로 부임하는데, 가족은 광주광역시 근처의 다른 지방으로 이사하는 경우)\n7.3.3. 가족여비는 가족 1명마다 다음 각 호의 구분에 따른 금액을 지급한다.\n가. 운임과 숙박비 : 본인이 여행하는 경우에 지급하는 금액\n나. 일비와 식비 : 본인이 여행하는 경우에 지급할 수 있는 금액을 기준으로 12세 이상인 가족에 대해서는 3분의 2, 12세 미만의 가족에 대해서는 3분의 1에 상당하는 금액\n7.4. 해외 가족여비', metadata={'source': 'mydata/출장여비규정.txt'}),
  Document(page_content='자를 말한다.\n15. 원격지 근무\n15.1. 출장의 정의에 부합하며, 지사, 사무소가 아닌 지역으로, 회사의 비용으로 숙박시설을 마련해 주는 경우를 원격지 근무라고 한다.\n15.2.원격지 근무자에게는 숙박비, 운임, 일비, 식비, 원격지근무수당을 지급한다.\n15.3. 숙박비는 숙박시설을 회사에서 제공(숙박시설 임차 및 숙박에 필요한 기본적인 물품 제공)하는 것으로 하여 별도로 지급하지 아니한다.\n15.4. 숙박에 필요한 기본 물품 구매는 별표9. 원격지 근무시 필요 물품 구매에 따른다.\n15.5. 운임은 월 4회(왕복 8회), 근무지에서 서울역까지의 KTX요금 혹은 이에 준하는 교통편의(시외버스 혹은 고속버스) 요금을 현금으로 지급한다.\n15.6. 일비와 식비는 월 4회(왕복 8회)의